# Création du dataset `imdb_wiki_corpus.csv`

Ce notebook permet de créer un corpus pour le fine-tuning d'un modèle pour la summarization. Il est créé à partir des corpus suivants : 

- des plots détaillés de Wikipédia : WikiPlots : https://github.com/markriedl/WikiPlots
- des synopsis courts de IMDB : Movie Synopsis Dataset : https://www.kaggle.com/datasets/linggarmaretva/movie-synopsis-dataset

A chaque plot détaillé, on lui associe son synopsis court IMDB en considérant que c'est son résumé. 

## Importation des modules

In [22]:
import re
import csv
from sklearn.model_selection import train_test_split

## Lecture des fichiers

In [23]:
# Fichiers avec les données
imdb_synopsis_file = "../movie_synopsis.csv"
wiki_plots_file = "../plots/plots"
wiki_titles_file = "../plots/titles"

# Lecture du fichier avec les synopsis IMDB
with open(imdb_synopsis_file, "r", encoding="utf-8") as csvfile:
    csv_reader = csv.DictReader(csvfile)
    
    # Création d'un dictionnaire {title: synopsis_IMDB}
    imdb_data = {row["title"].lower(): row["synopsis"] for row in csv_reader}
    
# Lecture du fichier avec les plots détaillés Wikipédia
with open(wiki_plots_file, "r", encoding="utf-8") as f:
    l = f.readline()
    
    plots = []
    temp_plot = [] # Un plot peut s'étaler sur plusieurs lignes
    
    while l:
        if l == "<EOS>\n": # Fin du plot
            plots.append("".join(temp_plot)) # Transformation de la liste en string
            temp_plot = [] # On vide la liste tampon
        else:
            temp_plot.append(l.strip())
    
        l = f.readline()
        
# Lecture du fichier avec les titres correspondants aux plots de Wikipédia
with open(wiki_titles_file, "r", encoding="utf-8") as f:
    wiki_titles = []
    l = f.readline()
    while l :
        title = re.sub(r"\((\w+|\s|\d+)+\)", "", l) # Suppression des infos entre parenthèses (ex : "(movie)", "(1998)", ...)
        wiki_titles.append(title.lower().strip()) # Tout en minuscules et suppression des trailing spaces
        l = f.readline()

## Création du dataset

In [24]:
# Liste de dictionnaires : un dico = une oeuvre = {"document": "long plot wikipédia", "summary": "court synopsis IMDB"}
oeuvres = [{"document": plot, "summary": imdb_data.get(title, "")} for plot, title in zip(plots, wiki_titles)]

print(f"En tout, on a {len(oeuvres)} oeuvres.")

En tout, on a 112936 oeuvres.


In [25]:
# Suppression des oeuvres pour lesquelles on a pas le synopsis IMDB
oeuvres_no_empty = [oeuvre for oeuvre in oeuvres if oeuvre["summary"]!=""]

print(f"{len(oeuvres_no_empty)} des {len(oeuvres)} plots Wikipédia ont un synopsis IMDB")

9807 des 112936 plots Wikipédia ont un synopsis IMDB


Note : Il est normal que notre dataset de plot wikipédia-synopsis IMDB soit plus long que celui de IMDB car il peut y avoir plusieurs fois la même oeuvre dans les plots alors que dans les synopsis, il y a un synopsis par oeuvre.

In [26]:
def write_dataset(data, path):
    """Ecrire le dataset dans un fichier CSV"""
    columns = data[0].keys() # Nom des colonnes ("document", "summary")
    with open(path, "w", encoding="utf-8", newline='') as output_file:
        dict_writer = csv.DictWriter(output_file, columns)
        dict_writer.writeheader()
        dict_writer.writerows(data)
        
write_dataset(oeuvres_no_empty, "../Data/imdb_wiki_corpus.csv")